**Download Data**\
The granular tick/trade data from DataBento can be downloaded from:
https://drive.google.com/file/d/1WE4YTNmtWPSvEsYBDD_V2lUYEE_J_sMJ/view?usp=sharing

It should be saved to the `data_dir` directory specified in the "Trade Strategy Variables" section below.

In [28]:
import pandas as pd


Below are variables for requesting data sets via DataBento API. Please use the example data from above instead of pulling from the API (there is a $ cost associated). These may not be needed once a database is established. We only want to pull the data once, when possible. Currently, these are only used in the code below for generating dynamic file names for outputting test data.

In [29]:
# Data Set Variables for DataBento; these may not be needed once a database is established
dataset = "GLBX.MDP3"
symbol = "NQZ3"
schema = "trades"
start_date = "2023-12-01T00:00:00"
end_date = "2023-12-31T23:59:59"
stype_in = "raw_symbol"


**Trade Strategy Variables**

Currently, these variables only apply to this specific strategy. They would need to be made customizable and toggleable in the future when developing new strategies. See code comments for individual variable descriptions.


**Data Source(s) and Storage**

These can be changed as needed. `data_dir` is the source data and `temp_data_dir` is used throughout the code below to store temporary output for debugging (the same output you see in the Jupyter notebook cells).

In [30]:
# Trade Strategy Variables
trade_tf = "15min" # The timeframe the trade is framed on. Use 'min' instead of 'm' for minutes because of df.resample() below
entry_tf = "1min" # The timeframe the trade is executed on
data_open_hour = 4 # The hour to begin the data feed for the trading day. Likely earlier than the trade window open in order to identify the earliest setups. (24-hour)
data_close_hour = 17 # The hour to end the data feed for the trading day. At minimum, the hour that all trades should be closed for the day. (24-hour)
tw_open_hour = 8 # The hour that the active trade window opens (24-hour)
tw_close_hour = 16 # The hour that the active trade window closes (24-hour)
stop_loss = 12 # Stop Loss (in points)
profit_target = 10 # Profit Target (in points)

# Data Source(s) and Storage
data_dir = "./data" # Where the original granular data is stored locally
temp_data_dir = "./data/temp" # Directory to store temp data for verification purposes


**Risk Management Variables**

These will typically be used for backtesting the majority of strategies. These may change based on the python backtesting library we end up using. Descriptions for each can be found in the code comments.

In [31]:
# Position Size and Risk Management Variables
starting_balance = 3000 # Starting account balance for simulation
scaling_interval = 5000 # Interval amount to add `scaling_position_size` from below
position_size = 1 # How many contracts/shares of the symbol
scaling_position_size = 1 # How many contracts/shares to add based on `scaling_interval` above
stop_loss = 12.00 # Number of total points for static stop loss
profit_target = 10.00 # Number of total points for static take profit target

Load test data into a pandas dataframe, convert time to US/Eastern, and set the index to `ts_recv` which is the time that each trade is finally executed on the exchange.

In [32]:
# Create a dataframe from DataBento .csv file for the most granular data
# **This will need changed to a database connection in the future**
df = pd.read_csv(f"{data_dir}/{symbol}_DataBento_{schema}_Dec_2023.csv")

# Format and convert to US Eastern time
# "ts_recv" is the sum of "ts_event" plus "ts_in_delta" in case it is not already calculated in the data
df["ts_recv"] = pd.to_datetime(df["ts_recv"], utc=True, dayfirst=False).dt.tz_convert("America/New_York")

# Set the index column
df.set_index("ts_recv", inplace=True)

# Format and output the original, granular dataframe after indexing
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df)

                                                                ts_event  rtype  publisher_id  instrument_id action side  depth     price  size  flags  ts_in_delta   sequence symbol
ts_recv                                                                                                                                                                              
2023-11-30 19:00:00.003574803-05:00  2023-12-01 00:00:00.003140441+00:00      0             1         260937      T    A      0  15964.25     1      0        15774  177919674   NQZ3
2023-11-30 19:00:00.517622404-05:00  2023-12-01 00:00:00.517401231+00:00      0             1         260937      T    B      0  15964.25     1      0        16315  177920136   NQZ3
2023-11-30 19:00:00.793528846-05:00  2023-12-01 00:00:00.793302653+00:00      0             1         260937      T    A      0  15963.75     1      0        16472  177920265   NQZ3
2023-11-30 19:00:00.793974319-05:00  2023-12-01 00:00:00.793741049+00:00      0           

Resampling the dataframe above into 30-second and 5-minute Open, High, Low, and Close data points (OHLC). These 4 data points (OHLC) make up a candlestick on a trading chart.

The data being output in this code block is also being saved to the `temp_data_dir` for easier browsing of the data for verification and troubleshooting.

<span style="color:yellow">***The resampled data generated directly below is manually verified to be correct and accurate***</span>

In [33]:
# Resample granular data into aggregated intervals for the strategy
# -------------------------------
# trade_tf candlesticks (OHLC)
df_trade_tf = df.resample(f"{trade_tf}")["price"].ohlc().ffill()

# Format and display the resampled trade timeframe output
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"\n######################\n{trade_tf} resampled data:\n######################\n {df_trade_tf}")
    
# Save resampled trade timeframe data to csv for verification purposes, if needed
df_trade_tf.to_csv(f"{temp_data_dir}/{symbol}_{trade_tf}_aggregated_ohlc.csv")
print(f"{trade_tf} aggregated data saved to: '{temp_data_dir}/{symbol}_{trade_tf}_aggregated_ohlc.csv'")


# entry_tf candlesticks (OHLC)
df_entry_tf = df.resample(f"{entry_tf}")["price"].ohlc().ffill()

# Format and display the resampled entry timeframe output
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"\n#####################\n{entry_tf} resampled data:\n#####################\n {df_entry_tf}")
    
# Save resampled entry timeframe data to csv for verification purposes, if needed
df_entry_tf.to_csv(f"{temp_data_dir}/{symbol}_{entry_tf}_aggregated_ohlc.csv")
print(f"{entry_tf} aggregated data saved to: '{temp_data_dir}/{symbol}_{entry_tf}_aggregated_ohlc.csv'")



######################
15min resampled data:
######################
                                open      high       low     close
ts_recv                                                          
2023-11-30 19:00:00-05:00  15964.25  15964.25  15958.25  15963.25
2023-11-30 19:15:00-05:00  15962.75  15963.75  15959.50  15961.50
2023-11-30 19:30:00-05:00  15961.25  15961.25  15956.75  15959.50
2023-11-30 19:45:00-05:00  15959.75  15959.75  15955.00  15958.75
2023-11-30 20:00:00-05:00  15958.75  15962.25  15953.50  15954.25
...                             ...       ...       ...       ...
2023-12-15 08:15:00-05:00  16592.00  16595.25  16584.00  16595.25
2023-12-15 08:30:00-05:00  16596.75  16603.50  16549.25  16567.75
2023-12-15 08:45:00-05:00  16565.25  16580.00  16560.50  16580.00
2023-12-15 09:00:00-05:00  16584.00  16601.00  16576.75  16598.75
2023-12-15 09:15:00-05:00  16598.75  16612.50  16521.50  16558.00

[1402 rows x 4 columns]
15min aggregated data saved to: './data/temp/NQ

Here, we are applying the defined data window variables from the strategy rules to the previously resampled data so that only the data within the desired data window is analyzed. There may be a much better way of doing this, I'm not sure.


The reason there is a data_window and trading_window is because the data needs to be analyzed prior to the trading window in case there is a valid trade entry as soon as the trading window opens (it happens quite often). For example, the trading strategy begins trading at 8AM. There may be a setup that allows for an entry at exactly 8AM. If the data window was the same as the trading window, that trade would be missed because it would not start looking for new swing highs and lows until after 8AM.


Perhaps just applying this time window to the original dataframe ***before*** resampling into the 30-sec and 5-min data is likely better and faster/more efficient.


<span style="color:yellow">***The filtered data generated directly below is manually verified to be correct and accurate***</span>

In [34]:
# Define the data window. In many cases, this will need to be larger than the active trading window to be sure that the earliest setups are identified.
def is_data_window(row):
    if isinstance(row.name, pd.Timestamp):
        return row.name.hour >= data_open_hour and row.name.hour < data_close_hour and row.name.dayofweek <= 4
    elif isinstance(row.name, tuple):
        return row.name[1].hour >= data_open_hour and row.name[1].hour < data_close_hour and row.name[1].dayofweek <= 4
    else:
        raise ValueError("Unexpected data type for row.name")

# Apply 'is_data_window' filter to granular data
df["is_data_window"] = df.apply(is_data_window, axis=1)

# Filter (copy) the results and only display rows where 'is_data_window' is True
df = df[df["is_data_window"]].copy()

# Display the filtered granular output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_data_window' to granular DataFrame and filtering results:\n {df}")

# Apply 'is_data_window' filter to entry timeframe resampled data
df_entry_tf["is_data_window"] = df_entry_tf.apply(is_data_window, axis=1)

# Filter (copy) the results and only display rows where 'is_data_window' is True
df_entry_tf = df_entry_tf[df_entry_tf["is_data_window"]].copy()

# Display the filtered entry timeframe resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_data_window' to {entry_tf} resampled DataFrame and filtering results:\n {df_entry_tf}")

# Apply 'is_data_window' filter to trade timeframe resampled data   
df_trade_tf["is_data_window"] = df_trade_tf.apply(is_data_window, axis=1)

# Filter (copy) the results and only display rows where 'is_data_window' is True
df_trade_tf = df_trade_tf[df_trade_tf["is_data_window"]].copy()

# Display the filtered trade timeframe resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_data_window' to {trade_tf} resampled DataFrame and filtering results:\n {df_trade_tf}")


Applying 'is_data_window' to granular DataFrame and filtering results:
                                                                 ts_event  rtype  publisher_id  instrument_id action side  depth     price  size  flags  ts_in_delta   sequence symbol  is_data_window
ts_recv                                                                                                                                                                                              
2023-12-01 04:00:00.007095156-05:00  2023-12-01 09:00:00.006874529+00:00      0             1         260937      T    A      0  15981.25     1      0        15470  181627916   NQZ3            True
2023-12-01 04:00:00.030049105-05:00  2023-12-01 09:00:00.029849545+00:00      0             1         260937      T    B      0  15981.75     1      0        15192  181627987   NQZ3            True
2023-12-01 04:00:00.030548991-05:00  2023-12-01 09:00:00.030358007+00:00      0             1         260937      T    B      0  15981.7

At this point, all of the data is available in the necessary aggregated timeframes for the trading strategy and we can begin to look for Swing High and Swing Low candlestick patterns within the 5-minute OHLC candlestick data.


The code below identifies all swing highs and lows and only fills the most recent swing that was generated (whether or not it was broken doesn't matter here).


A "swing high" is a three candlestick pattern where the 1st and 3rd candle highs are lower (and not equal to) the high of the 2nd candle. A "swing low" is a three candlestick pattern where the 1st and 3rd candle lows are higher (and not equal to) the low of the 2nd candle.

<span style="color:yellow">***The data output directly below is manually verified to be correct and accurate***</span>

In [35]:
# Calculate the rolling swing highs and swing lows over the last three, 5-minute candlesticks
df_trade_tf["SwingHigh"] = df_trade_tf["high"].shift(2).where((df_trade_tf["high"].shift(2) > df_trade_tf["high"].shift(1)) & (df_trade_tf["high"].shift(2) > df_trade_tf["high"].shift(3))).shift(-1)
df_trade_tf["SwingLow"] = df_trade_tf["low"].shift(2).where((df_trade_tf["low"].shift(2) < df_trade_tf["low"].shift(1)) & (df_trade_tf["low"].shift(2) < df_trade_tf["low"].shift(3))).shift(-1)

# # Forward-fill NaNs in df_trade_tf
# df_trade_tf["SwingHigh"] = df_trade_tf["SwingHigh"].ffill()
# df_trade_tf["SwingLow"] = df_trade_tf["SwingLow"].ffill()

with pd.option_context("display.min_rows", 20, "display.max_columns", None, "display.width", 1000):
    print(df_trade_tf)


                               open      high       low     close  is_data_window  SwingHigh  SwingLow
ts_recv                                                                                               
2023-12-01 04:00:00-05:00  15981.25  15988.75  15978.00  15988.00            True        NaN       NaN
2023-12-01 04:15:00-05:00  15987.75  15994.50  15986.50  15993.75            True        NaN       NaN
2023-12-01 04:30:00-05:00  15993.50  16000.75  15993.00  15997.00            True        NaN       NaN
2023-12-01 04:45:00-05:00  15997.25  15997.75  15983.00  15983.75            True   16000.75       NaN
2023-12-01 05:00:00-05:00  15983.50  15987.00  15981.25  15984.50            True        NaN       NaN
2023-12-01 05:15:00-05:00  15984.50  15987.00  15974.25  15978.25            True        NaN       NaN
2023-12-01 05:30:00-05:00  15978.75  15980.50  15969.75  15972.75            True        NaN       NaN
2023-12-01 05:45:00-05:00  15973.00  15977.00  15966.50  15966.50        

Once the most recent swing highs and lows within the 5-minute data are being identified, we are waiting for price to move above the most recent swing high (`BreakHigh`) or below the most recent swing low (`BreakLow`).

**TO-DO:**
  - If `BreakHigh` or `BreakLow` are 'True', then reset `SwingHigh` and/or `SwingLow` to the next most recent swing high or low that has not been broken (currently stuck on this part here a little bit)

<span style="color:red">**PLEASE NOTE:** This output has not yet been verified to ensure that it's correct and valid.</span>

In [36]:
# Check for the "break" of the most recent swing high and/or swing low. For example, price needs to move above a swing high or below a swing low and cannot be equal to.
df_trade_tf['BreakHigh'] = (df_trade_tf['high'] > df_trade_tf['SwingHigh'])
df_trade_tf['BreakLow'] = (df_trade_tf['low'] < df_trade_tf['SwingLow'])

with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df_trade_tf)
    
# If `BreakHigh` or `BreakLow` are 'True', then reset `SwingHigh` and/or `SwingLow` to the next most recent swing high or low that has not been broken

                               open      high       low     close  is_data_window  SwingHigh  SwingLow  BreakHigh  BreakLow
ts_recv                                                                                                                    
2023-12-01 04:00:00-05:00  15981.25  15988.75  15978.00  15988.00            True        NaN       NaN      False     False
2023-12-01 04:15:00-05:00  15987.75  15994.50  15986.50  15993.75            True        NaN       NaN      False     False
2023-12-01 04:30:00-05:00  15993.50  16000.75  15993.00  15997.00            True        NaN       NaN      False     False
2023-12-01 04:45:00-05:00  15997.25  15997.75  15983.00  15983.75            True   16000.75       NaN      False     False
2023-12-01 05:00:00-05:00  15983.50  15987.00  15981.25  15984.50            True        NaN       NaN      False     False
...                             ...       ...       ...       ...             ...        ...       ...        ...       ...
2023-12-

Then, once a swing high or low has been broken, we check for the next candle to close ***in the opposite direction*** of the swing high or low that was broken.

For example, if a swing high was broken (price moved higher than the swing high), it will wait for a red (or down-close) candle (close < open) to close in the 5-minute data and return `true` in the dataframe for `DownClose`. If a swing low was broken (price moved lower than the swing low), it will wait for a green (or up-close) candle (close > open) to close in the 5-minute data and return `true` in the dataframe for `UpClose`.

<span style="color:red">**PLEASE NOTE:** This output has not yet been verified to ensure that it's correct and valid.</span>

***This appears to be correctly identifying when a swing high or low is broken and when an opposite close candle happens, but we need to update the dataframe as these columns become invalidated. Currently stuck on this part a bit.***

In [37]:
# Check for a candle to close in the opposite direction of the broken swing high/low
df_trade_tf['DownClose'] = (df_trade_tf['close'] < df_trade_tf['open']) & df_trade_tf['BreakHigh'].shift(1)
df_trade_tf['UpClose'] = (df_trade_tf['close'] > df_trade_tf['open']) & df_trade_tf['BreakLow'].shift(1)

with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df_trade_tf)

                               open      high       low     close  is_data_window  SwingHigh  SwingLow  BreakHigh  BreakLow  DownClose  UpClose
ts_recv                                                                                                                                        
2023-12-01 04:00:00-05:00  15981.25  15988.75  15978.00  15988.00            True        NaN       NaN      False     False      False    False
2023-12-01 04:15:00-05:00  15987.75  15994.50  15986.50  15993.75            True        NaN       NaN      False     False      False    False
2023-12-01 04:30:00-05:00  15993.50  16000.75  15993.00  15997.00            True        NaN       NaN      False     False      False    False
2023-12-01 04:45:00-05:00  15997.25  15997.75  15983.00  15983.75            True   16000.75       NaN      False     False      False    False
2023-12-01 05:00:00-05:00  15983.50  15987.00  15981.25  15984.50            True        NaN       NaN      False     False      False  

Here, we are applying the defined trading window variables from the strategy rules to the previously resampled data so that only the data within the desired trade window is analyzed. I'm sure there is likely a much better way of doing this and the data_window block further above.


The reason there is a data_window and trading_window is because the data needs to be analyzed prior to the trading window in case there is a valid trade entry as soon as the trading window opens (it happens quite often). For example, the trading strategy begins trading at 8AM. There may be a setup that allows for an entry at exactly 8AM. If the data window was the same as the trading window, that trade would be missed because it would not start looking for new swing highs and lows until after 8AM.


Perhaps just applying this time window to the original dataframe ***before*** resampling into the 30-sec and 5-min data is likely better and faster/more efficient.

<span style="color:red">**PLEASE NOTE:** This output has not yet been verified to ensure that it's correct and valid.</span>

In [38]:
# Define the trading window to filter data; this may move into a separated functions file in the future as it would likely be used often between strategies
def is_trading_window(row):
    if isinstance(row.name, pd.Timestamp):
        return row.name.hour >= tw_open_hour and row.name.hour < tw_close_hour and row.name.dayofweek <= 4
    elif isinstance(row.name, tuple):
        return row.name[1].hour >= tw_open_hour and row.name[1].hour < tw_close_hour and row.name[1].dayofweek <= 4
    else:
        raise ValueError("Unexpected data type for row.name")

# Apply 'is_trading_window' filter to granular data
df["is_trading_window"] = df.apply(is_trading_window, axis=1)
# Filter (copy) the results and only display rows where 'is_trading_window' is True
df = df[df["is_trading_window"]].copy()
# Display the filtered granular output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_trading_window' to granular DataFrame and filtering results:\n {df}")

# Apply 'is_trading_window' filter to entry timeframe resampled data
df_entry_tf["is_trading_window"] = df_entry_tf.apply(is_trading_window, axis=1)
# Filter (copy) the results and only display rows where 'is_trading_window' is True
df_entry_tf = df_entry_tf[df_entry_tf["is_trading_window"]].copy()
# Display the filtered entry timeframe resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_trading_window' to {entry_tf} resampled DataFrame and filtering results:\n {df_entry_tf}")

# Apply 'is_trading_window' filter to trade timeframe resampled data   
df_trade_tf["is_trading_window"] = df_trade_tf.apply(is_trading_window, axis=1)
# Filter (copy) the results and only display rows where 'is_trading_window' is True
df_trade_tf = df_trade_tf[df_trade_tf["is_trading_window"]].copy()
# Display the filtered trade timeframe resampled output for verification
with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(f"Applying 'is_trading_window' to {trade_tf} resampled DataFrame and filtering results:\n {df_trade_tf}")


Applying 'is_trading_window' to granular DataFrame and filtering results:
                                                                 ts_event  rtype  publisher_id  instrument_id action side  depth     price  size  flags  ts_in_delta   sequence symbol  is_data_window  is_trading_window
ts_recv                                                                                                                                                                                                                 
2023-12-01 08:00:00.041508761-05:00  2023-12-01 13:00:00.041330717+00:00      0             1         260937      T    B      0  15924.50     1      0        15953  184385641   NQZ3            True               True
2023-12-01 08:00:00.152503179-05:00  2023-12-01 13:00:00.152250041+00:00      0             1         260937      T    B      0  15924.75     2      0        15875  184385741   NQZ3            True               True
2023-12-01 08:00:00.152578527-05:00  2023-12-01 13:00:00.

Just outputting the current 5-minute OHLC dataframe for verification/troubleshooting.

In [39]:
# Output the current entry_tf DataFrame to csv for troubleshooting
df_trade_tf.to_csv(f'{temp_data_dir}/{symbol}_{entry_tf}_sniper_trade_results.csv')
print(f"Current {entry_tf} DataFrame saved to .csv at '{temp_data_dir}/{symbol}_{entry_tf}_sniper_trade_results.csv' \n\n") 

with pd.option_context("display.min_rows", 10, "display.max_columns", None, "display.width", 1000):
    print(df_trade_tf)

Current 1min DataFrame saved to .csv at './data/temp/NQZ3_1min_sniper_trade_results.csv' 


                               open      high       low     close  is_data_window  SwingHigh  SwingLow  BreakHigh  BreakLow  DownClose  UpClose  is_trading_window
ts_recv                                                                                                                                                           
2023-12-01 08:00:00-05:00  15924.50  15942.50  15923.00  15932.75            True        NaN  15919.25      False     False      False    False               True
2023-12-01 08:15:00-05:00  15932.50  15948.00  15929.75  15941.75            True        NaN       NaN      False     False      False    False               True
2023-12-01 08:30:00-05:00  15942.00  15947.75  15937.00  15943.00            True    15948.0       NaN      False     False      False    False               True
2023-12-01 08:45:00-05:00  15943.00  15945.75  15927.25  15930.50            True        NaN 

***The code below is a work in progress.***

This is code that I started with in the original iteration and have been refactoring as I go. Assume that this is nonsense, is not working correctly, and currently invalid.

In [40]:
########################################################################
# The code below is a work in progress
########################################################################

# Check for Bullish and Bearish Fair Value Gaps (FVG) using the most granular data (1-minute, currently)
df_entry_tf["BullFVG"] = (df_entry_tf["low"].shift(-1).where(df_trade_tf["UpClose"].shift(1) == True) > df_entry_tf["high"].shift(1).where(df_trade_tf["UpClose"].shift(1) == True)).ffill()
df_entry_tf["BearFVG"] = (df_entry_tf["high"].shift(-1).where(df_trade_tf["DownClose"].shift(1) == True) < df_entry_tf["low"].shift(1).where(df_trade_tf["DownClose"].shift(1) == True)).ffill()

# When a Fair Value Gap is created, note the entry price
df_entry_tf["LongEntry"] = (df_entry_tf["low"].shift(-1).where(df_entry_tf["BullFVG"] == True)).shift(1)
df_entry_tf["ShortEntry"] = (df_entry_tf["high"].shift(-1).where(df_entry_tf["BearFVG"] == True)).shift(1)

# Determine stop loss and profit target prices and add them to the dataframe
df_entry_tf["LongStopLoss"] = (df_entry_tf["LongEntry"] - stop_loss)
df_entry_tf["LongProfitTarget"] = (df_entry_tf["LongEntry"] + profit_target)
df_entry_tf["ShortStopLoss"] = (df_entry_tf["ShortEntry"] + stop_loss)
df_entry_tf["ShortProfitTarget"] = (df_entry_tf["ShortEntry"] - profit_target)

# Output the DataFrame with swing high, swing low, BreakHigh, BreakLow, DownClose, and UpClose signals
with pd.option_context("display.max_rows", 20, "display.max_columns", None, "display.width", 1000):
    print(df_entry_tf)
    
# Output the entry_tf results to .csv
df_entry_tf.to_csv(f"{temp_data_dir}/{symbol}_30s_sniper_trade_results.csv")


                               open      high       low     close  is_data_window  is_trading_window  BullFVG  BearFVG  LongEntry  ShortEntry  LongStopLoss  LongProfitTarget  ShortStopLoss  ShortProfitTarget
ts_recv                                                                                                                                                                                                        
2023-12-01 08:00:00-05:00  15924.50  15929.00  15923.00  15928.25            True               True    False    False        NaN         NaN           NaN               NaN            NaN                NaN
2023-12-01 08:01:00-05:00  15928.00  15931.50  15928.00  15930.00            True               True    False    False        NaN         NaN           NaN               NaN            NaN                NaN
2023-12-01 08:02:00-05:00  15929.75  15934.00  15928.75  15934.00            True               True    False    False        NaN         NaN           NaN             

#### Need to add backtesting library functions and code below. The following rules should be included:

* Trade Window: 08:00AM - 16:00PM EDT (completed)
* No new trade executions between 09:25AM - 09:35AM; open positions are fine to hold through the New York open at 09:30AM
* No new trades executed after 15:55PM, open positions are find to hold through the New York close at 16:00PM
* All positions should be closed by 16:09:59PM; this could be extended based on funding company or broker trading windows
* While a position is open, new trade setups and executions may be added in the same direction
* While a position is open, oppositing setups should be ignored, but still tracked in case the actual entry takes place after the open position closes
* Rules regarding news and economic numbers release times should be added ASAP once the historical results are generating successfully